In [1]:
import pandas as pd

In [2]:
## how this data is generated? it is not on  github
df_html = pd.read_parquet("../data/HTML_data.parquet")

In [3]:
df_html

,reference_id,domain_start_id,response_url,response_text
0,23884,3292,https://bianchi-industrial.it/products/linear-...,<!doctypehtml><html class=no-js lang=en-GB><me...
1,65092,1265,https://vilagrancha.com/en/1292-complete-clutc...,<!doctypehtml><html lang=en><meta charset=utf-...


In [4]:
df = df_html.head(50)

In [5]:
df

,reference_id,domain_start_id,response_url,response_text
0,23884,3292,https://bianchi-industrial.it/products/linear-...,<!doctypehtml><html class=no-js lang=en-GB><me...
1,65092,1265,https://vilagrancha.com/en/1292-complete-clutc...,<!doctypehtml><html lang=en><meta charset=utf-...


In [4]:
import os
import requests
import urllib3
from urllib.parse import urlparse, urljoin
from bs4 import BeautifulSoup

# Suppress SSL warnings globally
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [7]:
def extract_img_attributes(html):
    """
    Extracts attributes from all <img> tags in the provided HTML content.
    Returns a list of dictionaries, each containing attributes of an <img> tag.
    """
    soup = BeautifulSoup(html, 'lxml')
    img_tags = soup.find_all('img')
    img_data = [img.attrs for img in img_tags]  # List comprehension for brevity
    
    return img_data

In [32]:
def save_combined_html(df, output_file="../data/combined.html"):
    # Combine all response text into one HTML file
    with open(output_file, "w", encoding="utf-8") as file:
        for html_content in df["response_text"]:
            file.write(html_content)
            file.write("\n")  # Separate each HTML content by a newline for readability

In [33]:
def download_images_with_local_path(dict_list, base_url, download_folder="../data/images"):
    """
    Downloads images from URLs in img attributes list and stores them locally.
    Adds the local path of each downloaded image back to the dictionary.
    """
    os.makedirs(download_folder, exist_ok=True)
    default_headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36'}
    
    for index, img_data in enumerate(dict_list):
        img_url = img_data.get("src")
        
        # Convert relative URLs to absolute URLs
        if img_url and urlparse(img_url).scheme == "":
            img_url = urljoin(base_url, img_url)
            print(f"Converted relative URL to absolute: {img_url}")

        # Replace backslashes with forward slashes
        if img_url:
            img_url = img_url.replace("\\", "/")
        
        # Check if URL is valid and not a data URI
        if img_url and urlparse(img_url).scheme in ["http", "https"]:
            # Generate filename consistent with the original function
            img_name = f"{index}_{os.path.basename(urlparse(img_url).path)}"
            img_path = os.path.join(download_folder, img_name)
            
            # Download the image using requests with SSL verification disabled
            try:
                # Attempt to download with custom User-Agent on the first try
                response = requests.get(img_url, headers=default_headers, stream=True, verify=False)
                response.raise_for_status()  # Raise an error for HTTP issues
                with open(img_path, "wb") as img_file:
                    for chunk in response.iter_content(1024):
                        img_file.write(chunk)
                print(f"Downloaded: {img_name}")
                
                # Add the relative path to the dictionary
                img_data["local_path"] = os.path.relpath(img_path, start=download_folder)
                
            except Exception as e:
                # Print the error and skip to the next item
                print(f"Error downloading {img_url}: {e}")
                
        else:
            # Skip data URIs or invalid URLs
            if img_url and img_url.startswith("data:image"):
                print(f"Skipping item {index} - Data URI found.")
            else:
                print(f"Skipping item {index} - No valid image URL found.")

In [34]:
# Process each HTML and base URL from the DataFrame
all_images_data = []

for idx, row in df.iterrows():
    html_content = row["response_text"]
    base_url = row["response_url"]
    
    # Extract image attributes for the current HTML page
    img_data = extract_img_attributes(html_content)
    
    # Download images and resolve paths
    download_images_with_local_path(img_data, base_url=base_url)
    
    # Add base URL and image data to the overall list
    for img in img_data:
        img["source_url"] = base_url  # Keep track of the page URL where the image was found
    all_images_data.extend(img_data)

# Optionally, convert all image data to a DataFrame for analysis
img_df = pd.DataFrame(all_images_data)

Downloaded: 0_cropped-Logo_hero.webp
Downloaded: 1_Logo_hero.webp
Skipping item 2 - Data URI found.
Skipping item 3 - Data URI found.
Downloaded: 4_pdf.svg
Downloaded: 5_pdf.svg
Downloaded: 6_pdf.svg
Downloaded: 7_pdf.svg
Downloaded: 8_pdf.svg
Downloaded: 9_pdf.svg
Downloaded: 10_pdf.svg
Downloaded: 11_pdf.svg
Downloaded: 12_cuscinetti_a_sfere-315x242.jpg
Downloaded: 13_trasmissioni-di-potenza-315x242.jpeg
Downloaded: 14_ICONA_LINEARE-315x242.jpg
Downloaded: 15_pneumatica_01-315x242.jpeg
Downloaded: 16_sistemi_di_tenuta-315x242.jpg
Downloaded: 17_monitoraggio_utensili-1-315x242.jpeg
Downloaded: 18_Gruppobianchi.png
Downloaded: 19_CQY_9.15_14.15_45.18_EN_RGB.png
Downloaded: 20_epda.webp
Downloaded: 21_FAI_I200_PAY-scaled.jpg
Downloaded: 22_privacy-by-pro.png
Downloaded: 0_2.jpg
Downloaded: 1_1.jpg
Downloaded: 2_2.jpg
Downloaded: 3_logo-1691485617.jpg
Downloaded: 4_logo-1691485617.jpg
Skipping item 5 - Data URI found.
Skipping item 6 - Data URI found.
Skipping item 7 - Data URI found.
Sk

In [35]:
def clean_image_folder(folder_path="../data/images"):
    """
    Deletes all files in the specified folder that are not .jpg files or contain 'logo' in the filename (case-insensitive).
    """
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        
        # Check if file is not a .jpg or contains "logo" (case insensitive)
        if not filename.lower().endswith(".jpg") or "logo" in filename.lower():
            try:
                os.remove(file_path)
                print(f"Deleted: {filename}")
            except Exception as e:
                print(f"Error deleting {filename}: {e}")

# Usage
clean_image_folder("../data/images")

Deleted: 0_0.gif
Deleted: 0_207ddfcc131f0070ff10.png
Deleted: 0_2406022251430.png
Deleted: 0_arkov-logo.png
Deleted: 0_bell-logo.png
Deleted: 0_Browse-Catalog-Icon.png
Deleted: 0_call.png
Deleted: 0_cropped-kavial30logo.png
Deleted: 0_cropped-Logo_hero.webp
Deleted: 0_eriks.svg
Deleted: 0_EXPERTSHOP.png
Deleted: 0_footer.png
Deleted: 0_g_logo_default_1
Deleted: 0_login.png
Deleted: 0_logo-bco.png
Deleted: 0_logo-full-group-white.png
Deleted: 0_logo.jpg
Deleted: 0_logo.png
Deleted: 0_logo.svg
Deleted: 0_logo_color_sin_fondo.png
Deleted: 0_logo_enriel_ONLINESTORE.png
Deleted: 0_logo_head.png
Deleted: 0_logo_kalydon.png
Deleted: 0_logo_kamatics.svg
Deleted: 0_not-found.svg
Deleted: 0_phone-red.svg
Deleted: 0_rodavelo_logohome.jpg
Deleted: 0_xlogorepuestosmurcia-768x164.png.pagespeed.ic.F2wS4jGxMa.png
Deleted: 0_საახალწლო-ნაძვის-ხეები-იკონკა1_ygas-28.png
Deleted: 100_20200406667_218.png
Deleted: 100_FAG-image108.png
Deleted: 100_pql9f04pwtiqhla4usr41k6iexavpszi.png
Deleted: 101_20200406347